In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
import torchvision
import numpy as np
import torchvision.transforms as transforms
import time

In [2]:
#Подготовка данных,приводим фотографии к единому размеру
#делаем аугументация данных путем вырезания случайного куска из фотографии 
data_directory = 'internship/internship_data/'
dataset = torchvision.datasets.ImageFolder(data_directory,
                transform=transforms.Compose([transforms.Resize(256),
                transforms.RandomCrop(227),transforms.ToTensor(),]))
#Разбиваем датасет на train 80% и val 20% для того чтобы отслеживать переобучение нейросети
N = (len(dataset))
train_set, val_set = torch.utils.data.random_split(dataset, [N-int(0.2*N), int(0.2*N)])

batch_size = 64
data_train = torch.utils.data.DataLoader(train_set, shuffle=True,batch_size=batch_size)
data_val = torch.utils.data.DataLoader(val_set, batch_size=batch_size)


In [3]:
#тренировка
def train(data_train , model , criterion, optimizer , epoch , use_cuda):
    model.train()
    train_loss = []
    for batch_idx, (inputs, targets) in enumerate(data_train):
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs , requires_grad=True), torch.autograd.Variable(targets)
        #прямой проход через нейросеть
        outputs = model(inputs)
        #вычисляем ошибку 
        loss = criterion(outputs, targets)
        train_loss.append(loss.item())
        #обнуляем градиенты
        optimizer.zero_grad()
        #вычисляем градиенты
        loss.backward()
        optimizer.step()

        
    return train_loss

#Тестируем нейросеть на данных которые она не видела при обучении
def test(val_loader, model, criterion, epoch, use_cuda):

    model.train(False)
    val_accuracy = []
    val_loss = []
    for batch_idx, (inputs, targets) in enumerate(val_loader):
        
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        val_loss.append(loss.item())
        prec1  = accuracy(outputs.data, targets.data)
        val_accuracy.append(prec1)
    return val_accuracy, val_loss

In [4]:

def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)
    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

In [7]:
#Сверточная нейросеть
class Net(nn.Module):
    
    def __init__(self):
        super(Net,self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3,96,kernel_size=7,stride=4),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3,stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(96,256,kernel_size=5,padding=2),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3,stride=2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(256,384,kernel_size=3,padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3,stride=2))
        self.fc1 = nn.Linear(13824,512)
        self.fc2 = nn.Linear(512,512)
        self.fc3 = nn.Linear(512,2)
        self.apply(weights_init)

    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0),-1)
        #print out.size()
        out = F.dropout(F.relu(self.fc1(out)))
        out = F.dropout(F.relu(self.fc2(out)))
        out = self.fc3(out)

        return out

#задаем начальеные веса
def weights_init(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        nn.init.normal_(m.weight, mean=0, std=1e-2)

In [8]:
#инициализируем модель
model = Net()

In [10]:
use_cuda = False
if torch.cuda.is_available():
    use_cuda = True
    model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad,model.parameters()) , lr = 0.0005, weight_decay=0.0005)

epoch_train_loss = []
epoch_val_loss = []
num_epochs = 15
for epoch in range(num_epochs):

    start_time = time.time()
    
    train_loss  = train(data_train, model, criterion, optimizer, epoch, use_cuda)
    val_accuracy , val_loss = test(data_val, model, criterion, epoch, use_cuda)
    if epoch>0:
        if np.mean(val_loss) < min(epoch_val_loss):
            torch.save(model, 'net_epoch_{}d.pt'.format(epoch))
    epoch_train_loss.append(np.mean(train_loss))
    epoch_val_loss.append(np.mean(val_loss))
    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))
    print("  training loss (in-iteration): \t{:.6f}".format(
        np.mean(train_loss)))
    print("  validation loss (in-iteration): \t{:.6f}".format(
        np.mean(val_loss)))
    print("  validation accuracy: \t\t\t{:.2f} %".format(
        np.mean(val_accuracy , axis=0).item()))
    

Epoch 1 of 15 took 340.434s
  training loss (in-iteration): 	0.145525
  validation loss (in-iteration): 	0.137453
  validation accuracy: 			94.44 %
Epoch 2 of 15 took 348.640s
  training loss (in-iteration): 	0.125637
  validation loss (in-iteration): 	0.122626
  validation accuracy: 			95.28 %
Epoch 3 of 15 took 347.512s
  training loss (in-iteration): 	0.115421
  validation loss (in-iteration): 	0.145900
  validation accuracy: 			94.17 %


KeyboardInterrupt: 